In [6]:
import tensorflow as tf

In [19]:
class CharCNNEmbedding:
    def __init__(self, config):
        self.char_vocab_size = config["char_vocab_size"]
        self.char_embedding_dim = config["char_embedding_dim"]

        self.kernel_sizes = config["kernel_sizes"]
        self.filter_size = config["elmo_hidden"] // len(self.kernel_sizes)

        self.seq_len = config["word_seq_len"]
        self.char_seq_len = config["char_seq_len"]
        
        with tf.variable_scope("char_cnn", reuse=tf.AUTO_REUSE):
            self.conv_filters = [
                tf.layers.Conv1D(self.filter_size, kernel_size)
                for kernel_size in self.kernel_sizes
            ]

        with tf.variable_scope("char_embedding", reuse=tf.AUTO_REUSE):
            self.embedding_weight = tf.get_variable("embedding_weight", 
                                        [self.char_vocab_size, self.char_embedding_dim],
                                        dtype=tf.float32)
            
            
    def forward(self, data):
        embed_input = tf.nn.embedding_lookup(self.embedding_weight, data["input"])

        conv_outputs = []
        conv_input = tf.reshape(embed_input, [-1, self.char_seq_len, self.char_embedding_dim])
        for conv, kernel_size in zip(self.conv_filters, self.kernel_sizes):
            conv_output = conv(conv_input)
            _conv_output = tf.reshape(conv_output, [-1, self.seq_len, conv_output.shape[1], self.filter_size])

            pool_output = tf.nn.max_pool(_conv_output, [1, 1, conv_output.shape[1], 1], [1, 1, 1, 1], 'VALID')
            pool_output = tf.squeeze(pool_output, axis=2)
            conv_outputs.append(pool_output)

        # shape = (batch_size, seq_len, embedding_dim)
        char_word_embedding = tf.concat(conv_outputs, axis=2)
        return char_word_embedding

In [20]:
class ELMO:
    def __init__(self, config):
        self.embedding = CharCNNEmbedding(config)
        self.hidden_size = config["elmo_hidden"]
        self.vocab_size = config["word_vocab_size"]
        self.seq_len = config["word_seq_len"]
        self.config = config
        
        with tf.variable_scope("elmo_rnn_cell"):
            self.forward_cell = tf.nn.rnn_cell.LSTMCell(self.hidden_size, reuse=tf.AUTO_REUSE)
            self.backward_cell = tf.nn.rnn_cell.LSTMCell(self.hidden_size, reuse=tf.AUTO_REUSE)
        
        #是否将输入concat到输出
        if config.get("use_skip_connection"):
            self.forward_cell = tf.nn.rnn_cell.ResidualWrapper(self.forward_cell)
            self.backward_cell = tf.nn.rnn_cell.ResidualWrapper(self.backward_cell)
            
        with tf.variable_scope("elmo_softmax"):
            softmax_weight_shape = [config["word_vocab_size"], config["elmo_hidden"]]

            self.forward_softmax_w = tf.get_variable("forward_softmax_w", softmax_weight_shape, dtype=tf.float32)
            self.backward_softmax_w = tf.get_variable("backward_softmax_w", softmax_weight_shape, dtype=tf.float32)

            self.forward_softmax_b = tf.get_variable("forward_softmax_b", [config["word_vocab_size"]])
            self.backward_softmax_b = tf.get_variable("backward_softmax_b", [config["word_vocab_size"]])
            
    def forward(self, data):
        embedding_output = self.embedding.forward(data)
        with tf.variable_scope("elmo_rnn_forward"):
            forward_outputs, forward_states = tf.nn.dynamic_rnn(self.forward_cell,
                                                                inputs=embedding_output,
                                                                sequence_length=data["input_len"],
                                                                dtype=tf.float32)

        with tf.variable_scope("elmo_rnn_backward"):
            backward_outputs, backward_states = tf.nn.dynamic_rnn(self.backward_cell,
                                                                  inputs=embedding_output,
                                                                  sequence_length=data["input_len"],
                                                                  dtype=tf.float32)

        # # Concatenate the forward and backward LSTM output
        forward_projection = tf.matmul(forward_outputs, tf.expand_dims(tf.transpose(self.forward_softmax_w), 0))
        forward_projection = tf.nn.bias_add(forward_projection, self.forward_softmax_b)

        backward_projection = tf.matmul(backward_outputs, tf.expand_dims(tf.transpose(self.backward_softmax_w), 0))
        backward_projection = tf.nn.bias_add(backward_projection, self.backward_softmax_b)

        return forward_outputs, backward_outputs, forward_projection, backward_projection
    
    
    def train(self, data, global_step_variable=None):
        forward_output, backward_output, _, _ = self.forward(data)

        forward_target = data["target"]
        forward_pred = tf.cast(tf.argmax(tf.nn.softmax(forward_output, -1), -1), tf.int32)
        forward_correct = tf.equal(forward_pred, forward_target)
        forward_padding = tf.sequence_mask(data["target_len"], maxlen=self.seq_len, dtype=tf.float32)

        forward_softmax_target = tf.cast(tf.reshape(forward_target, [-1, 1]), tf.int64)
        forward_softmax_input = tf.reshape(forward_output, [-1, self.hidden_size])
        forward_train_loss = tf.nn.sampled_softmax_loss(
            weights=self.forward_softmax_w, biases=self.forward_softmax_b,
            labels=forward_softmax_target, inputs=forward_softmax_input,
            num_sampled=self.config["softmax_sample_size"],
            num_classes=self.config["word_vocab_size"]
        )

        forward_train_loss = tf.reshape(forward_train_loss, [-1, self.seq_len])
        forward_train_loss = tf.multiply(forward_train_loss, forward_padding)
        forward_train_loss = tf.reduce_mean(forward_train_loss)

        backward_target = tf.reverse_sequence(data["target"], data["target_len"], seq_axis=1, batch_axis=0)
        backward_pred = tf.cast(tf.argmax(tf.nn.softmax(backward_output, -1), -1), tf.int32)
        backward_correct = tf.equal(backward_pred, backward_target)
        backward_padding = tf.sequence_mask(data["target_len"], maxlen=self.seq_len, dtype=tf.float32)

        backward_softmax_target = tf.cast(tf.reshape(backward_target, [-1, 1]), tf.int64)
        backward_softmax_input = tf.reshape(backward_output, [-1, self.hidden_size])
        backward_train_loss = tf.nn.sampled_softmax_loss(
            weights=self.backward_softmax_w, biases=self.backward_softmax_b,
            labels=backward_softmax_target, inputs=backward_softmax_input,
            num_sampled=self.config["softmax_sample_size"],
            num_classes=self.config["word_vocab_size"]
        )

        backward_train_loss = tf.reshape(backward_train_loss, [-1, self.seq_len])
        backward_train_loss = tf.multiply(backward_train_loss, backward_padding)
        backward_train_loss = tf.reduce_mean(backward_train_loss)

        train_loss = forward_train_loss + backward_train_loss
        train_correct = tf.concat([forward_correct, backward_correct], axis=-1)
        train_acc = tf.reduce_mean(tf.cast(train_correct, tf.float32))

        tf.summary.scalar("train_acc", train_acc)
        tf.summary.scalar("train_loss", train_loss)

        train_ops = tf.train.AdamOptimizer().minimize(train_loss)
        return train_loss, train_acc, train_ops

    def pred(self, data):
        elmo_projection_output = self.forward(data)
        eval_output = tf.nn.softmax(elmo_projection_output, dim=-1)
        return eval_output

In [21]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("-b", "--batch_size", type=int, default=1024)
parser.add_argument("-c", "--corpus_files", nargs='+', type=str,
                    default=["data/corpus/elmo.corpus.xlarge.1.txt"])

parser.add_argument("-e", "--epochs", type=int, default=10)
parser.add_argument("--verbose_freq", type=int, default=1)

parser.add_argument("--word_vocab_path", type=str, default="data/vocab/word.90k.vocab")
parser.add_argument("--char_vocab_path", type=str, default="data/vocab/jamo.100.vocab")

parser.add_argument("--word_seq_len", type=int, default=10)
parser.add_argument("--char_seq_len", type=int, default=7)

parser.add_argument("--char_embedding_dim", type=int, default=64)
parser.add_argument("--kernel_sizes", nargs='+', type=int, default=[1, 2, 3, 4])
parser.add_argument("--filter_sizes", nargs='+', type=int, default=None)

parser.add_argument("--elmo_hidden", type=int, default=512)
parser.add_argument("--softmax_sample_size", type=int, default=8196)

parser.add_argument("--prefetch_size", type=int, default=1024)

parser.add_argument("--log_dir", type=str, default="logs/")
parser.add_argument("--save_freq", type=int, default=1000)
parser.add_argument("--model_save_path", type=str, default="output/elmo.model.test")
parser.add_argument("--log_file_prefix", type=str, default="elmo.log")
args = parser.parse_known_args()[0]
config_dict = vars(args)

print(config_dict)

{'batch_size': 1024, 'corpus_files': ['data/corpus/elmo.corpus.xlarge.1.txt'], 'epochs': 10, 'verbose_freq': 1, 'word_vocab_path': 'data/vocab/word.90k.vocab', 'char_vocab_path': 'data/vocab/jamo.100.vocab', 'word_seq_len': 10, 'char_seq_len': 7, 'char_embedding_dim': 64, 'kernel_sizes': [1, 2, 3, 4], 'filter_sizes': None, 'elmo_hidden': 512, 'softmax_sample_size': 8196, 'prefetch_size': 1024, 'log_dir': 'logs/', 'save_freq': 1000, 'model_save_path': 'output/elmo.model.test', 'log_file_prefix': 'elmo.log'}


In [22]:
from han2jamo import Han2Jamo
from vocab_builder import CharWordVocab, WordVocab


class ElmoKoreanDataset:
    def __init__(self, config):
        self.corpus_files = config["corpus_files"]
        self.jamo_processor = Han2Jamo()

        self.char_vocab = CharWordVocab.load_vocab(config["char_vocab_path"])
        self.word_vocab = WordVocab.load_vocab(config["word_vocab_path"])

        self.seq_len = config["word_seq_len"]
        self.char_seq_len = config["char_seq_len"]
        self.corpus_size = self.get_corpus_size()
        print("Dataset Size:", self.corpus_size)

        config["char_vocab_size"] = len(self.char_vocab)
        config["word_vocab_size"] = len(self.word_vocab)

    def text_to_char_sequence(self, text):
        jamo_text = self.jamo_processor.str_to_jamo(text)
        char_idx_seq, seq_len = self.char_vocab.to_seq(jamo_text,
                                                       char_seq_len=self.char_seq_len,
                                                       seq_len=self.seq_len,
                                                       with_len=True)
        seq_len = self.seq_len if seq_len > self.seq_len else seq_len
        return char_idx_seq, seq_len

    def text_to_word_sequence(self, text):
        word_idx_seq, seq_len = self.word_vocab.to_seq(text, seq_len=self.seq_len + 1, with_len=True, with_eos=True)
        seq_len = self.seq_len + 1 if seq_len > self.seq_len + 1 else seq_len
        word_idx_seq, seq_len = word_idx_seq[1:], seq_len - 1
        return word_idx_seq, seq_len

    def produce_data(self, text):
        text = text.strip()
        char_word_input, input_len = self.text_to_char_sequence(text)
        word_target, target_len = self.text_to_word_sequence(text)

        return {"input": char_word_input, "input_len": input_len,
                "target": word_target, "target_len": target_len}

    def data_generator(self):
        for file_path in self.corpus_files:
            with open(file_path, "r", encoding="utf-8") as f:
                for text in f:
                    yield self.produce_data(text)

    def get_corpus_size(self):
        count = 0
        for file_path in self.corpus_files:
            with open(file_path) as file:
                count += sum(1 for _ in file)
        return count

In [24]:
import os
import time

import numpy as np
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Input, SpatialDropout1D
from keras.layers import LSTM, CuDNNLSTM, Activation
from keras.layers import Lambda, Embedding, Conv2D, GlobalMaxPool1D
from keras.layers import add, concatenate
from keras.layers.wrappers import TimeDistributed
from keras.models import Model, load_model
from keras.optimizers import Adagrad
from keras.constraints import MinMaxNorm
from keras.utils import to_categorical

from data import MODELS_DIR
from .custom_layers import TimestepDropout, Camouflage, Highway, SampledSoftmax


class ELMo(object):
    def __init__(self, parameters):
        self._model = None
        self._elmo_model = None
        self.parameters = parameters
        self.compile_elmo()

    def __del__(self):
        K.clear_session()
        del self._model

    def char_level_token_encoder(self):
        charset_size = self.parameters['charset_size']
        char_embedding_size = self.parameters['char_embedding_size']
        token_embedding_size = self.parameters['hidden_units_size']
        n_highway_layers = self.parameters['n_highway_layers']
        filters = self.parameters['cnn_filters']
        token_maxlen = self.parameters['token_maxlen']

        # Input Layer, word characters (samples, words, character_indices)
        inputs = Input(shape=(None, token_maxlen,), dtype='int32')
        # Embed characters (samples, words, characters, character embedding)
        embeds = Embedding(input_dim=charset_size, output_dim=char_embedding_size)(inputs)
        token_embeds = []
        # Apply multi-filter 2D convolutions + 1D MaxPooling + tanh
        for (window_size, filters_size) in filters:
            convs = Conv2D(filters=filters_size, kernel_size=[window_size, char_embedding_size], strides=(1, 1),
                           padding="same")(embeds)
            convs = TimeDistributed(GlobalMaxPool1D())(convs)
            convs = Activation('tanh')(convs)
            convs = Camouflage(mask_value=0)(inputs=[convs, inputs])
            token_embeds.append(convs)
        token_embeds = concatenate(token_embeds)
        # Apply highways networks
        for i in range(n_highway_layers):
            token_embeds = TimeDistributed(Highway())(token_embeds)
            token_embeds = Camouflage(mask_value=0)(inputs=[token_embeds, inputs])
        # Project to token embedding dimensionality
        token_embeds = TimeDistributed(Dense(units=token_embedding_size, activation='linear'))(token_embeds)
        token_embeds = Camouflage(mask_value=0)(inputs=[token_embeds, inputs])

        token_encoder = Model(inputs=inputs, outputs=token_embeds, name='token_encoding')
        return token_encoder

    def compile_elmo(self, print_summary=False):
        """
        Compiles a Language Model RNN based on the given parameters
        """

        if self.parameters['token_encoding'] == 'word':
            # Train word embeddings from scratch
            word_inputs = Input(shape=(None,), name='word_indices', dtype='int32')
            embeddings = Embedding(self.parameters['vocab_size'], self.parameters['hidden_units_size'], trainable=True, name='token_encoding')
            inputs = embeddings(word_inputs)

            # Token embeddings for Input
            drop_inputs = SpatialDropout1D(self.parameters['dropout_rate'])(inputs)
            lstm_inputs = TimestepDropout(self.parameters['word_dropout_rate'])(drop_inputs)

            # Pass outputs as inputs to apply sampled softmax
            next_ids = Input(shape=(None, 1), name='next_ids', dtype='float32')
            previous_ids = Input(shape=(None, 1), name='previous_ids', dtype='float32')
        elif self.parameters['token_encoding'] == 'char':
            # Train character-level representation
            word_inputs = Input(shape=(None, self.parameters['token_maxlen'],), dtype='int32', name='char_indices')
            inputs = self.char_level_token_encoder()(word_inputs)

            # Token embeddings for Input
            drop_inputs = SpatialDropout1D(self.parameters['dropout_rate'])(inputs)
            lstm_inputs = TimestepDropout(self.parameters['word_dropout_rate'])(drop_inputs)

            # Pass outputs as inputs to apply sampled softmax
            next_ids = Input(shape=(None, 1), name='next_ids', dtype='float32')
            previous_ids = Input(shape=(None, 1), name='previous_ids', dtype='float32')

        # Reversed input for backward LSTMs
        re_lstm_inputs = Lambda(function=ELMo.reverse)(lstm_inputs)
        mask = Lambda(function=ELMo.reverse)(drop_inputs)

        # Forward LSTMs
        for i in range(self.parameters['n_lstm_layers']):
            if self.parameters['cuDNN']:
                lstm = CuDNNLSTM(units=self.parameters['lstm_units_size'], return_sequences=True,
                                 kernel_constraint=MinMaxNorm(-1*self.parameters['cell_clip'],
                                                              self.parameters['cell_clip']),
                                 recurrent_constraint=MinMaxNorm(-1*self.parameters['cell_clip'],
                                                                 self.parameters['cell_clip']))(lstm_inputs)
            else:
                lstm = LSTM(units=self.parameters['lstm_units_size'], return_sequences=True, activation="tanh",
                            recurrent_activation='sigmoid',
                            kernel_constraint=MinMaxNorm(-1 * self.parameters['cell_clip'],
                                                         self.parameters['cell_clip']),
                            recurrent_constraint=MinMaxNorm(-1 * self.parameters['cell_clip'],
                                                            self.parameters['cell_clip'])
                            )(lstm_inputs)
            lstm = Camouflage(mask_value=0)(inputs=[lstm, drop_inputs])
            # Projection to hidden_units_size
            proj = TimeDistributed(Dense(self.parameters['hidden_units_size'], activation='linear',
                                         kernel_constraint=MinMaxNorm(-1 * self.parameters['proj_clip'],
                                                                      self.parameters['proj_clip'])
                                         ))(lstm)
            # Merge Bi-LSTMs feature vectors with the previous ones
            lstm_inputs = add([proj, lstm_inputs], name='f_block_{}'.format(i + 1))
            # Apply variational drop-out between BI-LSTM layers
            lstm_inputs = SpatialDropout1D(self.parameters['dropout_rate'])(lstm_inputs)

        # Backward LSTMs
        for i in range(self.parameters['n_lstm_layers']):
            if self.parameters['cuDNN']:
                re_lstm = CuDNNLSTM(units=self.parameters['lstm_units_size'], return_sequences=True,
                                    kernel_constraint=MinMaxNorm(-1*self.parameters['cell_clip'],
                                                                 self.parameters['cell_clip']),
                                    recurrent_constraint=MinMaxNorm(-1*self.parameters['cell_clip'],
                                                                    self.parameters['cell_clip']))(re_lstm_inputs)
            else:
                re_lstm = LSTM(units=self.parameters['lstm_units_size'], return_sequences=True, activation='tanh',
                               recurrent_activation='sigmoid',
                               kernel_constraint=MinMaxNorm(-1 * self.parameters['cell_clip'],
                                                            self.parameters['cell_clip']),
                               recurrent_constraint=MinMaxNorm(-1 * self.parameters['cell_clip'],
                                                               self.parameters['cell_clip'])
                               )(re_lstm_inputs)
            re_lstm = Camouflage(mask_value=0)(inputs=[re_lstm, mask])
            # Projection to hidden_units_size
            re_proj = TimeDistributed(Dense(self.parameters['hidden_units_size'], activation='linear',
                                            kernel_constraint=MinMaxNorm(-1 * self.parameters['proj_clip'],
                                                                         self.parameters['proj_clip'])
                                            ))(re_lstm)
            # Merge Bi-LSTMs feature vectors with the previous ones
            re_lstm_inputs = add([re_proj, re_lstm_inputs], name='b_block_{}'.format(i + 1))
            # Apply variational drop-out between BI-LSTM layers
            re_lstm_inputs = SpatialDropout1D(self.parameters['dropout_rate'])(re_lstm_inputs)

        # Reverse backward LSTMs' outputs = Make it forward again
        re_lstm_inputs = Lambda(function=ELMo.reverse, name="reverse")(re_lstm_inputs)

        # Project to Vocabulary with Sampled Softmax
        sampled_softmax = SampledSoftmax(num_classes=self.parameters['vocab_size'],
                                         num_sampled=int(self.parameters['num_sampled']),
                                         tied_to=embeddings if self.parameters['weight_tying']
                                         and self.parameters['token_encoding'] == 'word' else None)
        outputs = sampled_softmax([lstm_inputs, next_ids])
        re_outputs = sampled_softmax([re_lstm_inputs, previous_ids])

        self._model = Model(inputs=[word_inputs, next_ids, previous_ids],
                            outputs=[outputs, re_outputs])
        self._model.compile(optimizer=Adagrad(lr=self.parameters['lr'], clipvalue=self.parameters['clip_value']),
                            loss=None)
        if print_summary:
            self._model.summary()

    def train(self, train_data, valid_data):

        # Add callbacks (early stopping, model checkpoint)
        weights_file = os.path.join(MODELS_DIR, "elmo_best_weights.hdf5")
        save_best_model = ModelCheckpoint(filepath=weights_file, monitor='val_loss', verbose=1,
                                          save_best_only=True, mode='auto')
        early_stopping = EarlyStopping(patience=self.parameters['patience'], restore_best_weights=True)

        t_start = time.time()

        # Fit Model
        self._model.fit_generator(train_data,
                                  validation_data=valid_data,
                                  epochs=self.parameters['epochs'],
                                  workers=self.parameters['n_threads']
                                  if self.parameters['n_threads'] else os.cpu_count(),
                                  use_multiprocessing=True
                                  if self.parameters['multi_processing'] else False,
                                  callbacks=[save_best_model])

        print('Training took {0} sec'.format(str(time.time() - t_start)))

    def evaluate(self, test_data):

        def unpad(x, y_true, y_pred):
            y_true_unpad = []
            y_pred_unpad = []
            for i, x_i in enumerate(x):
                for j, x_ij in enumerate(x_i):
                    if x_ij == 0:
                        y_true_unpad.append(y_true[i][:j])
                        y_pred_unpad.append(y_pred[i][:j])
                        break
            return np.asarray(y_true_unpad), np.asarray(y_pred_unpad)

        # Generate samples
        x, y_true_forward, y_true_backward = [], [], []
        for i in range(len(test_data)):
            test_batch = test_data[i][0]
            x.extend(test_batch[0])
            y_true_forward.extend(test_batch[1])
            y_true_backward.extend(test_batch[2])
        x = np.asarray(x)
        y_true_forward = np.asarray(y_true_forward)
        y_true_backward = np.asarray(y_true_backward)

        # Predict outputs
        y_pred_forward, y_pred_backward = self._model.predict([x, y_true_forward, y_true_backward])

        # Unpad sequences
        y_true_forward, y_pred_forward = unpad(x, y_true_forward, y_pred_forward)
        y_true_backward, y_pred_backward = unpad(x, y_true_backward, y_pred_backward)

        # Compute and print perplexity
        print('Forward Langauge Model Perplexity: {}'.format(ELMo.perplexity(y_pred_forward, y_true_forward)))
        print('Backward Langauge Model Perplexity: {}'.format(ELMo.perplexity(y_pred_backward, y_true_backward)))

    def wrap_multi_elmo_encoder(self, print_summary=False, save=False):
        """
        Wrap ELMo meta-model encoder, which returns an array of the 3 intermediate ELMo outputs
        :param print_summary: print a summary of the new architecture
        :param save: persist model
        :return: None
        """

        elmo_embeddings = list()
        elmo_embeddings.append(concatenate([self._model.get_layer('token_encoding').output, self._model.get_layer('token_encoding').output],
                                           name='elmo_embeddings_level_0'))
        for i in range(self.parameters['n_lstm_layers']):
            elmo_embeddings.append(concatenate([self._model.get_layer('f_block_{}'.format(i + 1)).output,
                                                Lambda(function=ELMo.reverse)
                                                (self._model.get_layer('b_block_{}'.format(i + 1)).output)],
                                               name='elmo_embeddings_level_{}'.format(i + 1)))

        camos = list()
        for i, elmo_embedding in enumerate(elmo_embeddings):
            camos.append(Camouflage(mask_value=0.0, name='camo_elmo_embeddings_level_{}'.format(i + 1))([elmo_embedding,
                                                                                                         self._model.get_layer(
                                                                                                             'token_encoding').output]))

        self._elmo_model = Model(inputs=[self._model.get_layer('word_indices').input], outputs=camos)

        if print_summary:
            self._elmo_model.summary()

        if save:
            self._elmo_model.save(os.path.join(MODELS_DIR, 'ELMo_Encoder.hd5'))
            print('ELMo Encoder saved successfully')

    def save(self, sampled_softmax=True):
        """
        Persist model in disk
        :param sampled_softmax: reload model using the full softmax function
        :return: None
        """
        if not sampled_softmax:
            self.parameters['num_sampled'] = self.parameters['vocab_size']
        self.compile_elmo()
        self._model.load_weights(os.path.join(MODELS_DIR, 'elmo_best_weights.hdf5'))
        self._model.save(os.path.join(MODELS_DIR, 'ELMo_LM_EVAL.hd5'))
        print('ELMo Language Model saved successfully')

    def load(self):
        self._model = load_model(os.path.join(MODELS_DIR, 'ELMo_LM.h5'),
                                 custom_objects={'TimestepDropout': TimestepDropout,
                                                 'Camouflage': Camouflage})

    def load_elmo_encoder(self):
        self._elmo_model = load_model(os.path.join(MODELS_DIR, 'ELMo_Encoder.hd5'),
                                      custom_objects={'TimestepDropout': TimestepDropout,
                                                      'Camouflage': Camouflage})

    def get_outputs(self, test_data, output_type='word', state='last'):
        """
       Wrap ELMo meta-model encoder, which returns an array of the 3 intermediate ELMo outputs
       :param test_data: data generator
       :param output_type: "word" for word vectors or "sentence" for sentence vectors
       :param state: 'last' for 2nd LSTMs outputs or 'mean' for mean-pooling over inputs, 1st LSTMs and 2nd LSTMs
       :return: None
       """
        # Generate samples
        x = []
        for i in range(len(test_data)):
            test_batch = test_data[i][0]
            x.extend(test_batch[0])

        preds = np.asarray(self._elmo_model.predict(np.asarray(x)))
        if state == 'last':
            elmo_vectors = preds[-1]
        else:
            elmo_vectors = np.mean(preds, axis=0)

        if output_type == 'words':
            return elmo_vectors
        else:
            return np.mean(elmo_vectors, axis=1)

    @staticmethod
    def reverse(inputs, axes=1):
        return K.reverse(inputs, axes=axes)

    @staticmethod
    def perplexity(y_pred, y_true):

        cross_entropies = []
        for y_pred_seq, y_true_seq in zip(y_pred, y_true):
            # Reshape targets to one-hot vectors
            y_true_seq = to_categorical(y_true_seq, y_pred_seq.shape[-1])
            # Compute cross_entropy for sentence words
            cross_entropy = K.categorical_crossentropy(K.tf.convert_to_tensor(y_true_seq, dtype=K.tf.float32),
                                                       K.tf.convert_to_tensor(y_pred_seq, dtype=K.tf.float32))
            cross_entropies.extend(cross_entropy.eval(session=K.get_session()))

        # Compute mean cross_entropy and perplexity
        cross_entropy = np.mean(np.asarray(cross_entropies), axis=-1)

        return pow(2.0, cross_entropy)

Using TensorFlow backend.


ImportError: cannot import name 'MODELS_DIR'